In [1]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [2]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

price_change_percentage_24h  price_change_percentage_7d  \
coin_id                                                                    
bitcoin                              1.08388                     7.60278   
ethereum                             0.22392                    10.38134   
tether                              -0.21173                     0.04935   
ripple                              -0.37819                    -0.60926   
bitcoin-cash                         2.90585                    17.09717   
binancecoin                          2.10423                    12.85511   
chainlink                           -0.23935                    20.69459   
cardano                              0.00322                    13.99302   
litecoin                            -0.06341                     6.60221   
bitcoin-cash-sv                      0.92530                     3.29641   

                 price_change_percentage_14d  price_change_percentage_30d  \
coin_id                                                                     
bitcoin                              6.57509                      7.67258   
ethereum                             4.80849                      0.13169   
tether                               0.00640                     -0.04237   
ripple                               2.24984                      0.23455   
bitcoin-cash                        14.75334                     15.74903   
binancecoin                          6.80688                      0.05865   
chainlink                            9.30098                    -11.21747   
cardano                              5.55476                     10.10553   
litecoin                             7.28931                      1.21662   
bitcoin-cash-sv                     -1.86656                      2.88926   

                 price_change_percentage_60d  price_change_percentage_200d  \
coin_id                                                                      
bitcoin                             -3.25185                      83.51840   
ethereum                           -12.88890                     186.77418   
tether                               0.28037                      -0.00542   
ripple                             -17.55245                      39.53888   
bitcoin-cash                       -13.71793                      21.66042   
binancecoin                         36.33486                     155.61937   
chainlink                          -43.69522                     403.22917   
cardano                            -22.84776                     264.51418   
litecoin                           -17.23960                      27.49919   
bitcoin-cash-sv                    -24.87434                       7.42562   

                 price_change_percentage_1y  
coin_id                                      
bitcoin                            37.51761  
ethereum                          101.96023  
tether                              0.01954  
ripple                            -16.60193  
bitcoin-cash                       14.49384  
binancecoin                        69.69195  
chainlink                         325.13186  
cardano                           156.09756  
litecoin                          -12.66408  
bitcoin-cash-sv                    93.73082

In [3]:
# Generate summary statistics
df_market_data.describe()

price_change_percentage_24h  price_change_percentage_7d  \
count                    41.000000                   41.000000   
mean                     -0.269686                    4.497147   
std                       2.694793                    6.375218   
min                     -13.527860                   -6.094560   
25%                      -0.608970                    0.047260   
50%                      -0.063410                    3.296410   
75%                       0.612090                    7.602780   
max                       4.840330                   20.694590   

       price_change_percentage_14d  price_change_percentage_30d  \
count                    41.000000                    41.000000   
mean                      0.185787                     1.545693   
std                       8.376939                    26.344218   
min                     -18.158900                   -34.705480   
25%                      -5.026620                   -10.438470   
50%                       0.109740                    -0.042370   
75%                       5.510740                     4.578130   
max                      24.239190                   140.795700   

       price_change_percentage_60d  price_change_percentage_200d  \
count                    41.000000                     41.000000   
mean                     -0.094119                    236.537432   
std                      47.365803                    435.225304   
min                     -44.822480                     -0.392100   
25%                     -25.907990                     21.660420   
50%                      -7.544550                     83.905200   
75%                       0.657260                    216.177610   
max                     223.064370                   2227.927820   

       price_change_percentage_1y  
count                   41.000000  
mean                   347.667956  
std                   1247.842884  
min                    -17.567530  
25%                      0.406170  
50%                     69.691950  
75%                    168.372510  
max                   7852.089700

In [4]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [5]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
scaled  = StandardScaler().fit_transform(df_market_data)

In [6]:
# Create a DataFrame with the scaled data
scaled_df = pd.DataFrame(scaled,columns=df_market_data.columns)

# Copy the crypto names from the original data
scaled_df["coin_id"] = df_market_data.index
# Set the coinid column as index
scaled_df = scaled_df.set_index('coin_id')

# Display sample data
scaled_df.head(3)

price_change_percentage_24h  price_change_percentage_7d  \
coin_id                                                             
bitcoin                      0.508529                    0.493193   
ethereum                     0.185446                    0.934445   
tether                       0.021774                   -0.706337   

          price_change_percentage_14d  price_change_percentage_30d  \
coin_id                                                              
bitcoin                      0.772200                     0.235460   
ethereum                     0.558692                    -0.054341   
tether                      -0.021680                    -0.061030   

          price_change_percentage_60d  price_change_percentage_200d  \
coin_id                                                               
bitcoin                     -0.067495                     -0.355953   
ethereum                    -0.273483                     -0.115759   
tether                       0.008005                     -0.550247   

          price_change_percentage_1y  
coin_id                               
bitcoin                    -0.251637  
ethereum                   -0.199352  
tether                     -0.282061

---

### Find the Best Value for k Using the Original Data.

In [7]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1,11))

In [8]:
# Create an empty list to store the inertia values
inertia  = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list
for i in k:
    model = KMeans(n_clusters=i, random_state=0)
    model.fit(scaled_df)
    inertia.append(model.inertia_)

C:\Users\pc\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\pc\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\pc\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\pc\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable 

In [9]:
# Create a dictionary with the data to plot the Elbow curve
elbow = {"k" :k, "inertia": inertia}

# Create a DataFrame with the data to plot the Elbow curve
elbow_df = pd.DataFrame(elbow)
elbow_df.head(3)

k     inertia
0  1  287.000000
1  2  195.820218
2  3  123.190482

In [10]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_curve = elbow_df.hvplot.line(x="k", y = "inertia")
elbow_curve


:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:**  k = 4 is the best value

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [11]:
# Initialize the K-Means model using the best value for k
kmodel = KMeans(n_clusters=4)

In [12]:
# Fit the K-Means model using the scaled data
kmodel.fit(scaled_df)

C:\Users\pc\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\pc\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


KMeans(n_clusters=4)

In [13]:
# Predict the clusters to group the cryptocurrencies using the scaled data
pre_cluster = model.predict(scaled_df)

# Print the resulting array of cluster values.
print(pre_cluster)

[5 5 8 8 3 5 5 5 5 1 0 8 8 3 1 1 8 1 5 8 0 5 8 8 8 8 1 0 5 6 1 1 2 1 9 7 4
 0 1 8 0]


In [14]:
# Create a copy of the DataFrame
df_copy=  scaled_df.copy()

In [15]:
# Add a new column to the DataFrame with the predicted clusters
df_copy["cluster"] = pre_cluster

# Display sample data
df_copy.head(3)

price_change_percentage_24h  price_change_percentage_7d  \
coin_id                                                             
bitcoin                      0.508529                    0.493193   
ethereum                     0.185446                    0.934445   
tether                       0.021774                   -0.706337   

          price_change_percentage_14d  price_change_percentage_30d  \
coin_id                                                              
bitcoin                      0.772200                     0.235460   
ethereum                     0.558692                    -0.054341   
tether                      -0.021680                    -0.061030   

          price_change_percentage_60d  price_change_percentage_200d  \
coin_id                                                               
bitcoin                     -0.067495                     -0.355953   
ethereum                    -0.273483                     -0.115759   
tether                       0.008005                     -0.550247   

          price_change_percentage_1y  cluster  
coin_id                                        
bitcoin                    -0.251637        5  
ethereum                   -0.199352        5  
tether                     -0.282061        8

In [16]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
cluster_scatter = df_copy.hvplot.scatter( x = "price_change_percentage_24h", y="price_change_percentage_7d", hover_color = ["coin_id"] )
cluster_scatter


:Scatter   [price_change_percentage_24h]   (price_change_percentage_7d)

---

### Optimize Clusters with Principal Component Analysis.

In [17]:
# Create a PCA model instance and set `n_components=3`.
pcamodel = PCA(n_components=3)

In [18]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
pcafit = pcamodel.fit_transform(scaled_df)
# View the first five rows of the DataFrame. 
pcafit[:5]

array([[-0.60066733,  0.84276006,  0.46159457],
       [-0.45826071,  0.45846566,  0.95287678],
       [-0.43306981, -0.16812638, -0.64175193],
       [-0.47183495, -0.22266008, -0.47905316],
       [-1.15779997,  2.04120919,  1.85971527]])

In [19]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
pcamodel.explained_variance_ratio_

array([0.3719856 , 0.34700813, 0.17603793])

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** 37 + 34 + 17 = 88  . the total explained variance is 88 

In [20]:
# Create a new DataFrame with the PCA data.
pca_df = pd.DataFrame(pcafit, columns = ["pc1","pc2","pc3"])
# Creating a DataFrame with the PCA data

# Copy the crypto names from the original data
pca_df ["coin_id"]= df_market_data.index

# Set the coinid column as index
pca_df = pca_df.set_index("coin_id")

# Display sample data
pca_df

pc1       pc2       pc3
coin_id                                           
bitcoin              -0.600667  0.842760  0.461595
ethereum             -0.458261  0.458466  0.952877
tether               -0.433070 -0.168126 -0.641752
ripple               -0.471835 -0.222660 -0.479053
bitcoin-cash         -1.157800  2.041209  1.859715
binancecoin          -0.516534  1.388377  0.804071
chainlink            -0.450711  0.517699  2.846143
cardano              -0.345600  0.729439  1.478013
litecoin             -0.649468  0.432165  0.600303
bitcoin-cash-sv      -0.759014 -0.201200 -0.217653
crypto-com-chain     -0.248198 -1.376252 -1.462026
usd-coin             -0.438408 -0.175337 -0.663388
eos                  -0.693425 -0.473815 -0.527597
monero                0.060499  2.909404  1.498571
tron                 -0.393352 -0.108192 -0.012756
tezos                -0.796176 -0.494409  1.082812
okb                   0.064075 -1.269825 -1.098829
stellar              -0.489015 -0.732719 -0.062543
cosmos               -0.306272  0.703415  1.714224
cdai                 -0.513528 -0.142802 -0.656566
neo                  -0.362120 -0.986914 -0.728752
wrapped-bitcoin      -0.604265  0.827398  0.439316
leo-token            -0.413296 -0.674115 -1.076628
huobi-token          -0.407483 -0.212507 -0.351426
nem                   0.608974  0.563532 -1.148742
binance-usd          -0.450211 -0.151019 -0.647401
iota                 -0.764665 -0.517886  0.204990
vechain              -0.556315 -1.938209 -1.261776
zcash                -0.425147  0.492976  1.058048
theta-token           2.676868 -0.013954 -1.965207
dash                 -0.613923 -0.479337  0.339565
ethereum-classic     -0.579924 -0.356334 -0.114942
ethlend               8.089018 -3.896891  2.301382
maker                -0.389045  0.165041  0.379414
havven                0.865762 -2.261882  0.275583
omisego               0.111675  0.428316 -1.205398
celsius-degree-token  4.792395  6.767679 -1.986985
ontology             -0.632355 -2.108117 -0.652227
ftx-token            -0.593142  0.021485  0.209911
true-usd             -0.458131 -0.135734 -0.635284
digibyte             -0.297910 -0.191126 -0.909602

---

### Find the Best Value for k Using the PCA Data

In [21]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1,11))

In [23]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
for i in k:
    model = KMeans(n_clusters=i, random_state=0)
    model.fit(pca_df)
    inertia.append(model.inertia_)

C:\Users\pc\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\pc\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\pc\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\pc\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable 

In [25]:
# Create a dictionary with the data to plot the Elbow curve
elbowpca ={"k" : k,"inertia" : inertia}
# Create a DataFrame with the data to plot the Elbow curve
elbowpca_df = pd.DataFrame(elbowpca)
elbowpca_df.head(3)

k     inertia
0  1  256.874086
1  2  165.901994
2  3   93.774626

In [26]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
pcaplot =  elbowpca_df.hvplot.line(x= "k", y = "inertia")
pcaplot

:Curve   [k]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** 4


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:**  no

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [27]:
# Initialize the K-Means model using the best value for k
pcamodel = KMeans(n_clusters = 4)

In [28]:
# Fit the K-Means model using the PCA data
pcamodel.fit(pca_df)

C:\Users\pc\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\pc\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


KMeans(n_clusters=4)

In [32]:
# Predict the clusters to group the cryptocurrencies using the PCA data
pre_pca = pcamodel.predict(pca_df)
# Print the resulting array of cluster values.
print(pre_pca)

[0 0 2 2 0 0 0 0 0 2 2 2 2 0 2 0 2 2 0 2 2 0 2 2 2 2 2 2 0 2 2 2 1 0 2 2 3
 2 2 2 2]


In [33]:
# Create a copy of the DataFrame with the PCA data
pca_copy = pca_df.copy()

# Add a new column to the DataFrame with the predicted clusters
pca_copy["Clusters"] = pre_pca

# Display sample data
pca_copy.head(3)

pc1       pc2       pc3  Clusters
coin_id                                         
bitcoin  -0.600667  0.842760  0.461595         0
ethereum -0.458261  0.458466  0.952877         0
tether   -0.433070 -0.168126 -0.641752         2

In [35]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
clusters_plot_pca = pca_copy.hvplot.scatter(
    x= "pc1",
    y= "pc2",
    by= "Clusters",
    hover_cols=["coin_id"])
   

clusters_plot_pca

:NdOverlay   [Clusters]
   :Scatter   [pc1]   (pc2,coin_id)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [36]:
# Composite plot to contrast the Elbow curves
pcaplot + elbow_curve

:Layout
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k]   (inertia)

In [37]:
# Composite plot to contrast the clusters
# YOUR CODE HERE!
clusters_plot_pca + cluster_scatter

:Layout
   .NdOverlay.I :NdOverlay   [Clusters]
      :Scatter   [pc1]   (pc2,coin_id)
   .Scatter.I   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:**  Using less clusters is better.